In [ ]:
%pip install PyGithub ipywidgets


In [ ]:
from github import Github
from ipywidgets import interact_manual, SelectMultiple, Checkbox
from getpass import getpass
from IPython.display import clear_output


def display_org_deleter():
    # Insert your token here
    token = getpass('Enter your GitHub token: ')

    # First create a Github instance:
    g = Github(token)

    # Get organizations
    orgs = g.get_user().get_orgs()
    org_names = [org.login for org in orgs]

    def delete_org(org_names, confirm_delete):
        for org_name in org_names:
            org = g.get_organization(org_name)

            if org.public_repos > 0 and not confirm_delete:
                print(
                    f"The organization {org_name} has public repositories. Please confirm deletion.")
                return

            # PyGithub does not support deleting an organization directly.
            # However, you can use the following workaround to delete an organization:
            org._requester.requestJsonAndCheck(
                "DELETE",
                org.url,
            )

            print(
                f"The organization {org_name} has been deleted successfully.")

        # Clear output and display widget again
        clear_output(wait=True)
        display_org_deleter()

    interact_widget = interact_manual(delete_org, org_names=SelectMultiple(
        options=org_names), confirm_delete=Checkbox(description='Confirm deletion'))
    return interact_widget.widget


display(display_org_deleter())
